<a href="https://colab.research.google.com/github/Aydin-ab/CV_DMTet/blob/main/CV_DMTet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing Dependencies: Kaolin and PyTorch3D

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
from pathlib import Path
INSTALL_PATH = Path("/content/drive/'My Drive'/CV_DMTet/")
%cd  $INSTALL_PATH

Mounted at /content/drive
/content/drive/My Drive/CV_DMTet


In [ ]:
# reinstall cython, install usd-core (for 3D rendering), and clone into kaolin repo
!pip uninstall Cython --yes
!pip install Cython==0.29.20  --quiet
!pip install usd-core --quiet
# !git clone --recursive https://github.com/NVIDIAGameWorks/kaolin
# %cd kaolin

Found existing installation: Cython 0.29.32
Uninstalling Cython-0.29.32:
  Successfully uninstalled Cython-0.29.32
     |████████████████████████████████| 2.0 MB 7.6 MB/s 
     |████████████████████████████████| 24.4 MB 665 kB/s 


In [ ]:
# installing kaolin and check version
%env IGNORE_TORCH_VER=1
%env KAOLIN_INSTALL_EXPERIMENTAL=1
KAOLIN_PATH = INSTALL_PATH / "kaolin"
%cd $INSTALL_PATH
!if [ ! -d $KAOLIN_PATH ]; then git clone --recursive https://github.com/NVIDIAGameWorks/kaolin; fi;
%cd $KAOLIN_PATH
SETUP_CHECK = KAOLIN_PATH / "kaolin" / "version.py"
!echo Checking if $SETUP_CHECK exists
!if [ ! -f $SETUP_CHECK ]; then python setup.py develop; fi;
!python -c "import kaolin; print(kaolin.__version__)"

env: IGNORE_TORCH_VER=1
env: KAOLIN_INSTALL_EXPERIMENTAL=1
/content/drive/My Drive/CV_DMTet
/content/drive/My Drive/CV_DMTet/kaolin
Checking if /content/drive/My Drive/CV_DMTet/kaolin/kaolin/version.py exists
0.12.0


In [ ]:
# installing PyTorch3D (for rendering)
import os
import sys
import torch
need_pytorch3d=False
try:
    import pytorch3d
except ModuleNotFoundError:
    need_pytorch3d=True
if need_pytorch3d:
    if torch.__version__.startswith("1.12.") and sys.platform.startswith("linux"):
        # We try to install PyTorch3D via a released wheel.
        pyt_version_str=torch.__version__.split("+")[0].replace(".", "")
        version_str="".join([
            f"py3{sys.version_info.minor}_cu",
            torch.version.cuda.replace(".",""),
            f"_pyt{pyt_version_str}"
        ])
        !pip install fvcore iopath
        !pip install --no-index --no-cache-dir pytorch3d -f https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/{version_str}/download.html
    else:
        # We try to install PyTorch3D from source.
        !curl -LO https://github.com/NVIDIA/cub/archive/1.10.0.tar.gz
        !tar xzf 1.10.0.tar.gz
        os.environ["CUB_HOME"] = os.getcwd() + "/cub-1.10.0"
        !pip install 'git+https://github.com/facebookresearch/pytorch3d.git@stable'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 50 kB 4.2 MB/s 
     |████████████████████████████████| 42 kB 1.1 MB/s 
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221122-py3-none-any.whl size=61484 sha256=dfa28825dfa80e4e699cfc0bc547f420f6d2f011486d514846ddad4e39f08176
  Stored in directory: /root/.cache/pip/wheels/2d/e4/d7/be0b4010933f5fffea6385e9b319eac9d6e56c82ee4a0164e5
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31547 sha256=6d4c3ffc11329df169a8120005053b92843566566dd03a507a63bc0bfbcc2687
  Stored in directory: /root/.cache/pip/wheels/aa/cc/ed/ca4e88beef656b01c84b9185196513ef2faf74a5a379b043a7
Successfully built fvcore iopath
Looking in links: https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/py37_cu113_pyt1121/download.html
     |████████████████████████████████| 47.2 MB 1.2 MB/s 


# Import packages

In [ ]:
import numpy as np
import torch
import kaolin

from pytorch3d.datasets import (
    R2N2,
    ShapeNetCore,
    collate_batched_meshes,
    render_cubified_voxels,
)
from pytorch3d.renderer import (
    FoVPerspectiveCameras,
    PointLights,
    RasterizationSettings,
    TexturesVertex,
    look_at_view_transform,
)

from kaolin.ops.conversions import (
    trianglemeshes_to_voxelgrids,
    marching_tetrahedra,
    voxelgrids_to_cubic_meshes,
    voxelgrids_to_trianglemeshes
)

from kaolin.io.shapenet import (
    ShapeNetV2
)

from pytorch3d.structures import Meshes
from torch.utils.data import DataLoader

!wget https://raw.githubusercontent.com/facebookresearch/pytorch3d/main/docs/tutorials/utils/plot_image_grid.py
from plot_image_grid import image_grid # rendering function

# add path for demo utils functions 
sys.path.append(os.path.abspath(''))

--2022-11-30 12:43:26--  https://raw.githubusercontent.com/facebookresearch/pytorch3d/main/docs/tutorials/utils/plot_image_grid.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1608 (1.6K) [text/plain]
Saving to: ‘plot_image_grid.py.1’

plot_image_grid.py. 100%[===================>]   1.57K  --.-KB/s    in 0.003s  

2022-11-30 12:43:26 (622 KB/s) - ‘plot_image_grid.py.1’ saved [1608/1608]



# Import Dataset: Subset of ShapeNetV2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    torch.cuda.set_device(device)
else:
    device = torch.device("cpu")
device

device(type='cuda', index=0)

In [ ]:
from pathlib import Path

In [ ]:
SHAPENET_PATH = "/content/drive/MyDrive/CV_DMTet/Core"
# SHAPENET_PATH = "/content/drive/MyDrive/FALL 2022/Computer Vision/Project/Core"
# SHAPENET_ZIP = Path(SHAPENET_PATH) / "ShapeNetCore.v2.zip"
# !curl "https://drive.google.com/drive/folders/1WKoRDtHdqubpHMam4iVOMe8D536KHBOY?usp=share_link" -o SHAPENET_PATH
SYNSETS_IDS = ['02747177', '02801938', '02808440', '02818832', '02828884', '02871439', '02876657', '02880940', '02924116', '02933112']
shapenet_train = ShapeNetV2(SHAPENET_PATH, categories=SYNSETS_IDS, output_dict=True)
shapenet_test = ShapeNetV2(SHAPENET_PATH, categories=SYNSETS_IDS, output_dict=True, train=False)

# Visualization of Train model & add 3D checkpoint

In [ ]:
"""
shapenet_train contains dict that stores model properties
"""

sample_model = shapenet_train[8] # change the index here for different models
sample_verts = sample_model['mesh'][0]
sample_faces = sample_model['mesh'][1]
sample_feats = sample_model['mesh'][4]
sample_id = sample_model['name'].split('/')[1]

print(sample_model)

{'mesh': return_type(vertices=tensor([[ 0.0734, -0.2761, -0.1726],
        [ 0.0601,  0.1218, -0.1775],
        [ 0.0601, -0.2761, -0.1775],
        ...,
        [-0.2201, -0.3234, -0.0937],
        [-0.2372, -0.3234, -0.0333],
        [-0.1880, -0.3234, -0.1476]]), faces=tensor([[   0,    1,    2],
        [   1,    0,    3],
        [   2,    1,    0],
        ...,
        [6817, 6814, 6815],
        [6814, 6817, 6816],
        [6818, 6816, 6817]]), uvs=tensor([[-1.0073,  2.6101],
        [-1.0104,  0.1351],
        [-1.0073,  0.1351],
        ...,
        [-2.4010,  2.6101],
        [-2.4010,  0.1351],
        [-2.3010,  2.6101]]), face_uvs_idx=tensor([[-1, -1, -1],
        [-1, -1, -1],
        [-1, -1, -1],
        ...,
        [-1, -1, -1],
        [-1, -1, -1],
        [-1, -1, -1]]), materials=[{'Kd': tensor([0.3373, 0.3373, 0.3373]), 'Ka': tensor([0., 0., 0.]), 'Ks': tensor([0.4000, 0.4000, 0.4000])}, {'Kd': tensor([0., 0., 0.]), 'Ka': tensor([0., 0., 0.]), 'Ks': tensor([0.400

In [ ]:
# Voxelization
res = 16
voxelgrids = trianglemeshes_to_voxelgrids(sample_verts.unsqueeze(0), sample_faces, resolution=res)

In [ ]:
# Can visualize model using Kaolin's 3D checkpoint

log_dir = "/content/drive/MyDrive/CV_DMTet/Logs"
timelapse = kaolin.visualize.Timelapse(log_dir)

In [ ]:
timelapse.add_mesh_batch(
    category='sample',
    faces_list=[sample_faces.cpu()],
    vertices_list=[sample_verts.cpu()],
)

In [ ]:
voxel_verts, voxel_faces = voxelgrids_to_cubic_meshes(voxelgrids.to(device))
voxel_verts, voxel_faces = voxel_verts[0], voxel_faces[0]

In [ ]:
timelapse.add_mesh_batch(
    category='sample_voxel',
    faces_list=[voxel_faces.cpu()],
    vertices_list=[voxel_verts.cpu()],
)

# Access Dash3D and checkpoints

In [ ]:
#Use pyngrok to access localhost:80 on Colab

!pip install pyngrok --quiet
from pyngrok import ngrok

# Terminate open tunnels if exist
ngrok.kill()
# Setting the authtoken (optional)
# Get authtoken from https://dashboard.ngrok.com/auth
NGROK_AUTH_TOKEN = "2IGctyaa9n7vRBd8qq7pzd0bNKh_2pDFmKRk5Af1QDq295xZ4"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

     |████████████████████████████████| 761 kB 7.6 MB/s 


In [ ]:
#generating a public url mapped to localhost 80
public_url = ngrok.connect(port=80, proto="http", options={"bind_tls": True, "local": True})
print("Tracking URL:", public_url)

Tracking URL: NgrokTunnel: "http://50c0-34-125-215-160.ngrok.io" -> "http://localhost:80"


In [ ]:
#Start Kaolin Dash3D on localhost:80
!kaolin-dash3d --logdir=/content/drive/MyDrive/CV_DMTet/Logs --port=80

/bin/bash: kaolin-dash3d: command not found


# Training

In [ ]:
"""
Data preprocessing: 

voxelize meshes into resolution of 100 first, then voxelize at lower res (~16) to thicken thin structures
After performing marching cubes on low res voxel grid, sample 3000 points from surfaces as input to the generator

Initialization: 

a uniform deformable tetrahedral grid generated by Quartet with resolution ~70 (DMTet)

Generator:

Receive (v, f(v)), where v is 3D coords, and f(v) is features of v (RGD, uvs, etc.) (Not considering for now)
Output a predicted s(v), sdf for v, and a deform vector [v1, v2, v3] to perform marching tetrahedra
Loss = Chamfer L2 + Laplacian regularization

A feature vector will also be output for surface refinement (Not considering for now)

"""

'\nData preprocessing: \n\nvoxelize meshes into resolution of 100 first, then voxelize at lower res (~16) to thicken thin structures\nAfter performing marching cubes on low res voxel grid, sample 3000 points from surfaces as input to the generator\n\nInitialization: \n\na uniform deformable tetrahedral grid generated by Quartet with resolution ~70 (DMTet)\n\nGenerator:\n\nReceive (v, f(v)), where v is 3D coords, and f(v) is features of v (RGD, uvs, etc.) (Not considering for now)\nOutput a predicted s(v), sdf for v, and a deform vector [v1, v2, v3] to perform marching tetrahedra\nLoss = Chamfer L2 + Laplacian regularization\n\nA feature vector will also be output for surface refinement (Not considering for now)\n\n'

In [ ]:
# Hyperparams
res=32 # resolution of voxelgrid
chamfer_sample_num=20000 # num of sampled point when evaluate chamfer distance
lr = 1e-3
laplacian_weight = 0.1
iterations = 5000
save_every = 100
multires = 2
grid_res = 128

In [ ]:
from kaolin.ops.mesh import (
    sample_points
)

from kaolin.metrics.pointcloud import (
    chamfer_distance
)

#randomly sample points from predicted mesh and ground truth mesh and calculate Chamfer distance
def get_Chamfer_L2(pred_verts, pred_faces, gt_verts, gt_faces):

    # generating point cloud using verts and faces

    pred_sample = sample_points(
        vertices=pred_verts.unsqueeze(0),
        faces=pred_faces,
        num_samples=chamfer_sample_num
    )

    gt_sample = sample_points(
        vertices=gt_verts.unsqueeze(0),
        faces=gt_faces,
        num_samples=chamfer_sample_num
    )

    pred_point_cloud = pred_sample[0].to(device)
    gt_point_cloud = gt_sample[0].to(device)

    # calculating chamfer distance
    dist = chamfer_distance(pred_point_cloud, gt_point_cloud)

    return dist

In [ ]:
d = get_Chamfer_L2(voxel_verts, voxel_faces, sample_verts, sample_faces)
print('Chamfer L2 distance between voxelization and ground truth: ', d)

Chamfer L2 distance between voxelization and ground truth:  tensor([229.1679], device='cuda:0')


In [ ]:
# Laplacian regularization using umbrella operator (Fujiwara / Desbrun).
# https://mgarland.org/class/geom04/material/smoothing.pdf
def laplace_regularizer_const(mesh_verts, mesh_faces):
    term = torch.zeros_like(mesh_verts)
    norm = torch.zeros_like(mesh_verts[..., 0:1])

    v0 = mesh_verts[mesh_faces[:, 0], :]
    v1 = mesh_verts[mesh_faces[:, 1], :]
    v2 = mesh_verts[mesh_faces[:, 2], :]

    term.scatter_add_(0, mesh_faces[:, 0:1].repeat(1,3), (v1 - v0) + (v2 - v0))
    term.scatter_add_(0, mesh_faces[:, 1:2].repeat(1,3), (v0 - v1) + (v2 - v1))
    term.scatter_add_(0, mesh_faces[:, 2:3].repeat(1,3), (v0 - v2) + (v1 - v2))

    two = torch.ones_like(v0) * 2.0
    norm.scatter_add_(0, mesh_faces[:, 0:1], two)
    norm.scatter_add_(0, mesh_faces[:, 1:2], two)
    norm.scatter_add_(0, mesh_faces[:, 2:3], two)

    term = term / torch.clamp(norm, min=1.0)

    return torch.mean(term**2)

def loss_f(mesh_verts, mesh_faces, points, it):
    pred_points = kaolin.ops.mesh.sample_points(mesh_verts.unsqueeze(0), mesh_faces, chamfer_sample_num)[0][0]
    chamfer = kaolin.metrics.pointcloud.chamfer_distance(pred_points.unsqueeze(0), points.unsqueeze(0)).mean()
    if it > iterations//2:
        lap = laplace_regularizer_const(mesh_verts, mesh_faces)
        return chamfer + lap * laplacian_weight
    return chamfer

In [ ]:
from tqdm import tqdm



# MLP + Positional Encoding
class Decoder(torch.nn.Module):
    def __init__(self, input_dims = 3, internal_dims = 128, output_dims = 4, hidden = 5, multires = 2):
        super().__init__()
        self.embed_fn = None
        if multires > 0:
            embed_fn, input_ch = get_embedder(multires)
            self.embed_fn = embed_fn
            input_dims = input_ch

        # net = (torch.nn.Linear(input_dims, internal_dims, bias=False), torch.nn.ReLU())
        # for i in range(hidden-1):
        #     net = net + (torch.nn.Linear(internal_dims, internal_dims, bias=False), torch.nn.ReLU())
        # net = net + (torch.nn.Linear(internal_dims, output_dims, bias=False),)
        # self.net = torch.nn.Sequential(*net)

        self.net = torch.nn.Sequential(
            torch.nn.Linear(input_dims, 256, bias=False),
            torch.nn.ReLU(),
            torch.nn.Linear(256, 256, bias=False),
            torch.nn.ReLU(),
            torch.nn.Linear(256, 128, bias=False),
            torch.nn.ReLU(),
            torch.nn.Linear(128, 64, bias=False),
            torch.nn.ReLU(),
            torch.nn.Linear(64, output_dims, bias=False),
        )

    def forward(self, p):
        if self.embed_fn is not None:
            p = self.embed_fn(p)
        out = self.net(p)
        return out

    def pre_train_sphere(self, iter):
        print ("Initialize SDF to sphere")
        loss_fn = torch.nn.MSELoss()
        optimizer = torch.optim.Adam(list(self.parameters()), lr=1e-4)

        for i in tqdm(range(iter)):
            p = torch.rand((1024,3), device='cuda') - 0.5
            ref_value  = torch.sqrt((p**2).sum(-1)) - 0.3
            output = self(p)
            loss = loss_fn(output[...,0], ref_value)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print("Pre-trained MLP", loss.item())


# Positional Encoding from https://github.com/yenchenlin/nerf-pytorch/blob/1f064835d2cca26e4df2d7d130daa39a8cee1795/run_nerf_helpers.py
class Embedder:
    def __init__(self, **kwargs):
        self.kwargs = kwargs
        self.create_embedding_fn()
        
    def create_embedding_fn(self):
        embed_fns = []
        d = self.kwargs['input_dims']
        out_dim = 0
        if self.kwargs['include_input']:
            embed_fns.append(lambda x : x)
            out_dim += d
            
        max_freq = self.kwargs['max_freq_log2']
        N_freqs = self.kwargs['num_freqs']
        
        if self.kwargs['log_sampling']:
            freq_bands = 2.**torch.linspace(0., max_freq, steps=N_freqs)
        else:
            freq_bands = torch.linspace(2.**0., 2.**max_freq, steps=N_freqs)
            
        for freq in freq_bands:
            for p_fn in self.kwargs['periodic_fns']:
                embed_fns.append(lambda x, p_fn=p_fn, freq=freq : p_fn(x * freq))
                out_dim += d
                    
        self.embed_fns = embed_fns
        self.out_dim = out_dim
        
    def embed(self, inputs):
        return torch.cat([fn(inputs) for fn in self.embed_fns], -1)

def get_embedder(multires):
    embed_kwargs = {
                'include_input' : True,
                'input_dims' : 3,
                'max_freq_log2' : multires-1,
                'num_freqs' : multires,
                'log_sampling' : True,
                'periodic_fns' : [torch.sin, torch.cos],
    }
    
    embedder_obj = Embedder(**embed_kwargs)
    embed = lambda x, eo=embedder_obj : eo.embed(x)
    return embed, embedder_obj.out_dim

In [ ]:
def SDF_train(epoch, model, trainset, optimizer):
    """
    for point cloud, extract 3D feature vectors by PVCNN and cat them into an R^1 vector F(v,x)
    one copy will be used here for prediction of SDF

    for voxel, extract 3D feature vector by sampling on surface

    initial prediction will consist of four layers with dimensions:
    256, 256, 128, 64

    losses will be Chamfer L2 distance between pred and gt

    check in kaolin's documentation for how to batch data
    """

    #dataset is a list of model ditionaries
    for idx, model_dict in enumerate(trainset):

      #get vertices and faces of meshes

      mesh = model_dict['mesh'] 
      verts = mesh[0]
      faces = mesh[1]

      #voxelize 3D meshes

      voxel_grid = trianglemeshes_to_voxelgrids(verts.unsqueeze(0), faces, resolution=res)

      